## AI for Medicine Course 1 Week 1 lecture exercises

# Patient Overlap and Data Leakage

Patient overlap in medical data is a part of a more general problem in machine learning called **data leakage**.  To identify patient overlap in this week's graded assignment, you'll check to see if a patient's ID appears in both the training set and the test set. You should also verify that you don't have patient overlap in the training and validation sets, which is what you'll do here.

Below is a simple example showing how you can check for and remove patient overlap in your training and validations sets.

In [1]:
# Import necessary packages
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import os
import seaborn as sns
sns.set()

### Read in the data from a csv file

First, you'll read in your training and validation datasets from csv files. Run the next two cells to read these csvs into `pandas` dataframes.

In [2]:
# Read csv file containing training data
train_df = pd.read_csv("nih/train-small.csv")
# Print first 5 rows
print(f'There are {train_df.shape[0]} rows and {train_df.shape[1]} columns in the training dataframe')
train_df.head()

There are 1000 rows and 16 columns in the training dataframe


,Image,Atelectasis,Cardiomegaly,Consolidation,Edema,Effusion,Emphysema,Fibrosis,Hernia,Infiltration,Mass,Nodule,PatientId,Pleural_Thickening,Pneumonia,Pneumothorax
0,00008270_015.png,0,0,0,0,0,0,0,0,0,0,0,8270,0,0,0
1,00029855_001.png,1,0,0,0,1,0,0,0,1,0,0,29855,0,0,0
2,00001297_000.png,0,0,0,0,0,0,0,0,0,0,0,1297,1,0,0
3,00012359_002.png,0,0,0,0,0,0,0,0,0,0,0,12359,0,0,0
4,00017951_001.png,0,0,0,0,0,0,0,0,1,0,0,17951,0,0,0


In [3]:
# Read csv file containing validation data
valid_df = pd.read_csv("nih/valid-small.csv")
# Print first 5 rows
print(f'There are {valid_df.shape[0]} rows and {valid_df.shape[1]} columns in the validation dataframe')
valid_df.head()

There are 200 rows and 16 columns in the validation dataframe


,Image,Atelectasis,Cardiomegaly,Consolidation,Edema,Effusion,Emphysema,Fibrosis,Hernia,Infiltration,Mass,Nodule,PatientId,Pleural_Thickening,Pneumonia,Pneumothorax
0,00008270_015.png,0,0,0,0,0,0,0,0,0,0,0,8270,0,0,0
1,00029855_001.png,1,0,0,0,1,0,0,0,1,0,0,29855,0,0,0
2,00001297_000.png,0,0,0,0,0,0,0,0,0,0,0,1297,1,0,0
3,00012359_002.png,0,0,0,0,0,0,0,0,0,0,0,12359,0,0,0
4,00017951_001.png,0,0,0,0,0,0,0,0,1,0,0,17951,0,0,0


### Extract and compare the PatientId columns from the train and validation sets
By running the next four cells you will do the following:
1. Extract patient IDs from the train and validation sets
2. Convert these arrays of numbers into `set()` datatypes for easy comparison
3. Identify patient overlap in the intersection of the two sets

In [4]:
# Extract patient id's for the training set
ids_train = train_df.PatientId.values
# Extract patient id's for the validation set
ids_valid = valid_df.PatientId.values

In [5]:
# Create a "set" datastructure of the training set id's to identify unique id's
ids_train_set = set(ids_train)
print(f'There are {len(ids_train_set)} unique Patient IDs in the training set')
# Create a "set" datastructure of the validation set id's to identify unique id's
ids_valid_set = set(ids_valid)
print(f'There are {len(ids_valid_set)} unique Patient IDs in the training set')

There are 928 unique Patient IDs in the training set
There are 199 unique Patient IDs in the training set


In [6]:
# Identify patient overlap by looking at the intersection between the sets
patient_overlap = list(ids_train_set.intersection(ids_valid_set))
n_overlap = len(patient_overlap)
print(f'There are {n_overlap} Patient IDs in both the training and validation sets')
print('')
print(f'These patients are in both the training and validation datasets:')
print(f'{patient_overlap}')

There are 197 Patient IDs in both the training and validation sets

These patients are in both the training and validation datasets:
[17920, 15360, 17417, 28682, 16397, 27665, 25628, 22559, 22047, 17951, 17443, 8741, 8230, 22572, 25645, 28208, 11315, 10294, 2614, 8760, 15929, 10808, 26685, 17470, 8767, 19014, 12359, 26185, 1610, 589, 17997, 8270, 13904, 27725, 18000, 20052, 12379, 25695, 13408, 27235, 4195, 11366, 6761, 22135, 26232, 121, 5241, 27260, 10877, 5759, 16007, 10888, 1168, 2705, 658, 25746, 9365, 11420, 8348, 16030, 29855, 16034, 12966, 29865, 28844, 3759, 9911, 14520, 9912, 26810, 1212, 12487, 26825, 1738, 21195, 28876, 30415, 1232, 720, 26835, 25813, 16604, 20188, 25315, 17126, 9447, 15078, 18669, 17135, 20213, 21750, 4344, 4858, 27900, 25858, 4360, 2312, 14603, 14604, 12045, 23309, 6926, 20240, 1297, 5904, 23825, 5905, 29454, 13592, 6426, 17691, 14107, 3868, 26399, 19233, 5410, 10531, 17702, 1831, 5418, 14125, 17714, 10035, 10548, 3382, 13111, 1849, 11579, 10557, 27455, 1

### Identify rows (indices) of overlapping patients and remove from either the train or validation set
Run the next two cells to do the following:
1. Create lists of the overlapping row numbers in both the training and validation sets. 
2. Drop the overlapping patient records from the validation set (could also choose to drop from train set)

In [7]:
train_overlap_idxs = []
valid_overlap_idxs = []
for idx in range(n_overlap):
    train_overlap_idxs.extend(train_df.index[train_df['PatientId'] == patient_overlap[idx]].tolist())
    valid_overlap_idxs.extend(valid_df.index[valid_df['PatientId'] == patient_overlap[idx]].tolist())
    
print(f'These are the indices of overlapping patients in the training set: ')
print(f'{train_overlap_idxs}')
print(f'These are the indices of overlapping patients in the validation set: ')
print(f'{valid_overlap_idxs}')

These are the indices of overlapping patients in the training set: 
[74, 112, 14, 557, 142, 187, 39, 91, 511, 598, 145, 164, 4, 176, 152, 50, 36, 193, 10, 24, 46, 604, 72, 51, 11, 95, 156, 49, 67, 635, 82, 3, 75, 124, 85, 45, 0, 63, 81, 134, 61, 163, 195, 177, 858, 7, 994, 131, 62, 171, 130, 13, 518, 150, 84, 143, 765, 21, 38, 80, 98, 148, 179, 109, 139, 87, 126, 170, 22, 1, 106, 26, 58, 190, 767, 149, 12, 157, 159, 181, 405, 100, 71, 107, 460, 894, 48, 97, 117, 146, 5, 122, 111, 128, 33, 65, 90, 44, 41, 115, 184, 6, 197, 296, 440, 789, 840, 985, 69, 123, 129, 962, 42, 160, 99, 191, 77, 101, 15, 27, 30, 110, 29, 2, 56, 102, 105, 196, 55, 57, 53, 104, 154, 138, 151, 17, 141, 878, 43, 136, 147, 64, 137, 92, 93, 178, 60, 207, 311, 674, 59, 28, 113, 68, 501, 9, 167, 18, 103, 34, 169, 32, 175, 78, 132, 168, 172, 108, 140, 189, 161, 114, 500, 125, 119, 8, 40, 54, 66, 89, 96, 31, 180, 37, 174, 16, 116, 183, 173, 94, 52, 249, 121, 182, 19, 79, 667, 165, 396, 88, 837, 118, 918, 155, 153, 185, 1

In [8]:
# Drop the overlapping rows from the validation set
valid_df.drop(valid_overlap_idxs, inplace=True)

### Check that everything worked as planned by rerunning the patient ID comparison between train and validation sets.

When you run the next two cells you should see that there are now fewer records in the validation set and that the overlap problem has been removed!

In [11]:
# Extract patient id's for the validation set
ids_valid = valid_df.PatientId.values
# Create a "set" datastructure of the validation set id's to identify unique id's
ids_valid_set = set(ids_valid)
print(f'There are {len(ids_valid_set)} unique Patient IDs in the validation set')

There are 2 unique Patient IDs in the validation set


In [10]:
# Identify patient overlap by looking at the intersection between the sets
patient_overlap = list(ids_train_set.intersection(ids_valid_set))
n_overlap = len(patient_overlap)
print(f'There are {n_overlap} Patient IDs in both the training and validation sets')

There are 0 Patient IDs in both the training and validation sets


### Congratulations! You removed overlapping patients from the validation set! 

You could have just as well removed them from the training set. 

Always be sure to check for patient overlap in your train, validation and test sets.